<a href="https://colab.research.google.com/github/Surojit-KB/UNIV-AI-ML2-HW2/blob/main/Copy_of_HW2_Part2_distribute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML-2: Trees, Model Interrogation and Bayesian Workflow
# Homework 2: Rossman Kaggle: Forecasting Sales
# Part 2 : Modelling without embeddings!
**ML-2 Cohort 1** <br>
**Instructor: Dr. Rahul Dave**<br>
**Max Score: 100** <br>

In [1]:
#importing libraries
import numpy as np
import scipy.stats
import scipy.special
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
from matplotlib import cm
import pandas as pd
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid
from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate
from keras.layers.embeddings import Embedding
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import pickle
import csv
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split

## Part 2: Modelling without Entity Embeddings

Remember the parameters we need to use

![Parameters.jpeg](https://drive.google.com/uc?export=view&id=1ROfqM3F5hWwJyrvQr_J1ATovNIW5niOs)

Lets import the feature_train_data.pickle file and set X,y values from the pickle file

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
with open("/content/drive/MyDrive/Colab Notebooks/UNIV-AI-ML2/HW/feature_train_data.pickle", 'rb') as f:
    (X,y) = pickle.load(f)

In [5]:
# we will split the train_ratio and 90% and 10% and set the train_size
train_ratio = 0.9
num_records = len(X)
train_size = int(train_ratio * num_records)

In [6]:
#lets look at our data
X[1], y[1]

(array([1058,    0,    0,    0,    1,    0,    0,    1]), '4491')

The next set of inputs is following:

1. Do you want to one hot encode the data?
2. Do you want to provide embeddings as input - this will be set to True for models with entity embeddings
3. Do you want to save the emmbeddings? - again set to true if you want to entity embeddings
4. if 3 is set to true, we want to save them to a embeddings.pickle


In [7]:
one_hot_as_input = True #one_hot is set to True
embeddings_as_input = False #embeddings later on needs to set to true for Part 3
save_embeddings = False
saved_embeddings_fname = "embeddings.pickle"  # set save_embeddings to True to create this file

### BASELINE MODEL WITHOUT ONE-HOT ENCODING 
TO UNDERSTAND HOW ONE-HOT ENCODING AFFECTS OUR RESULTS, I AM BUILDING A BASELINE RANDOM FOREST MODEL WITHOUT ANY ENCODING AND GETTING A BASELINE RESULTS. SINCE RANDOM FORESTS DO NOT REQUIRE THE VARIABLES TO BE ONE-HOT ENCODED, I EXCEPT IT TO PERFORM QUITE WELL

In [11]:
#WHILE SAVING THE DATA, WE HAD SAVED THE EARLIER DATA FIRST. HENCE WE CAN DIRECTLY SPLIT THE DATA ON THE BASIS OF INDICES
#WE WOULD WANT TO TRAIN ON EARLIER DATA AND PREDICT ON LATER DATA, BECAUSE THAT IS HOW OUR PRACTICAL USE CASE WILL BE.
X_train=X[:train_size,:]
X_val=X[train_size:,:]
y_train=y[:train_size]
y_val=y[train_size:]
y_train=np.log(y_train.astype(np.int))
y_val=np.log(y_val.astype(np.int))
y_max=max(np.max(y_train),np.max(y_val))
y_train=y_train/y_max
y_val=y_val/y_max

In [12]:
def sample(X, y, n):
    '''random samples'''
    num_row = X.shape[0]
    indices = np.random.randint(num_row, size=n)
    return X[indices, :], y[indices]

In [13]:
X_train, y_train = sample(X_train, y_train, 200000)  # Simulate data sparsity
print("Number of samples used for training: " + str(y_train.shape[0]))

Number of samples used for training: 200000


**Lets define MAPE first**

In [16]:
#defining mape
def MAPE(Y_actual,Y_Predicted):
    #your code here
    return np.mean(np.abs((Y_actual-Y_Predicted)/Y_actual))

In [14]:
#your code here
#MODEL WITHOUT ONE HOT ENCODING
model=RandomForestRegressor(n_estimators = 200, max_depth = 35, min_samples_split = 2, min_samples_leaf = 1,n_jobs=-1,verbose=1)
model.fit(X_train,y_train)
y_pred=model.predict(X_val)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   50.2s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   52.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    1.5s finished


In [18]:
print('MAPE score is:',MAPE(y_val,y_pred))
print('R2 score is:',r2_score(y_val,y_pred))

MAPE score is: 0.016132731892635584
R2 score is: 0.7786951516022276


### MODELS WITH ONE HOT ENCODING
HERE THERE ARE 2 PARTS:<BR>
1. IN THE FIRST PART, I WILL ONE-HOT ENCODE ALL THE FEATURES EXCEPT THE STORE INDEX FEATURE. THE STORE-INDEX FEATURE HAS HIGH CARDINALITY AND I WANT TO SEE THE EFFECT ONE-HOT ENCODING HAS ON IT.
2. IN THE SECOND PART, WE ARE GOING TO ONE-HOT ENCODE EVERYTHING AND OBSERVE THE RESULTS.

Define a function to one hot encode the training set and after split transform your training set using the function

In [20]:
#FUNCTION FOR ONE HOT ENCODING THE DATA
#THE STORE INDEX FEATURE HAS 1115 CATEGORIES WHICH IS HIGH CARDINALITY
#HENCE I AM FIRST TRYING TO MAKE A MODEL WHICH ONE-HOT ENCODES ALL THE OTHER FEATURES EXCEPT STORE INDEX
#THIS IS JUST TO UNDERSTAND HOW THE ENCODING AFFECTS THE FEATURES 
def hot_encode(data):
    #your code here
    encoder=OneHotEncoder()
    encoder.fit(data[:,1:])
    train_data=encoder.transform(data[:,1:])
    return train_data

Split the data into X_train, X_val, y_train, y_val based on the train_size

In [21]:
train_data=hot_encode(X)
train_data=np.column_stack((X[:,0].reshape(-1,1),train_data.todense()))

In [22]:
#your code here
#train_data=hot_encode(X)
X_train=train_data[:train_size,:]
X_val=train_data[train_size:,:]
y_train=y[:train_size]
y_val=y[train_size:]

In [23]:
#WE ARE USING LOF TRANSFORMED TARGET VARIABLE FOR ALL OF PUR MODELS
# THIS CELL LOG TRANSFORMS THE DATA AND SCALES IT AS WELL
y_train=np.log(y_train.astype(np.int))
y_val=np.log(y_val.astype(np.int))
y_max=max(np.max(y_train),np.max(y_val))
y_train=y_train/y_max
y_val=y_val/y_max

Lets also sample the data

**Why do we do this??**

WE ARE SAMPLING THE DATA TO INTRODUCE SPARSITY INTO THE DATA. WE HAVE VERY LESS FEATURES (7) AS COMPARED TO TOTAL SAMPLES (AROUND 10,00,000). HENCE TO INTRODUCE THE SPARSITY, WE ARE RANDOMLY SAMPLING 2,00,000 OUT OF THE WHOLE DATASET.

In [24]:
X_train, y_train = sample(X_train, y_train, 200000)  # Simulate data sparsity
print("Number of samples used for training: " + str(y_train.shape[0]))

Number of samples used for training: 200000


## Now lets work with Models without embedding!!

**We will be log-transforming the dependent variable(y) because it is long-tailed** - keep this in mind for each model or do the conversion after you split the data itself

### 2.1: Random Forests

1. Define a RandomForestRegressor model - with n_esitmators = 200, max_depth = 35, min_samples_split = 2, min_samples_leaf = 1
2. Fit on the training data
3. Predict on the validation and training data
4. evaluate the model - calculate the MAPE for validation and training data

**These parameters are from the paper** 

In [30]:
#Here we have all the features except store index one hot encoded
print('The shape of the training data is:',X_train.shape)

The shape of the training data is: (200000, 69)


In [25]:
#your code here
model=RandomForestRegressor(n_estimators = 200, max_depth = 35, min_samples_split = 2, min_samples_leaf = 1,n_jobs=-1,verbose=1)
model.fit(X_train,y_train)
y_pred=model.predict(X_val)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   56.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  4.3min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    1.5s finished


In [28]:
print('MAPE score is:',MAPE(y_val,y_pred))
print('R2 score is:',r2_score(y_val,y_pred))

MAPE score is: 0.014966925814039453
R2 score is: 0.8054359429306465


NOW, WE WILL ONE HOT ENCODE ALL THE FEATURES.

In [37]:
def hot_encode(data):
    #your code here
    encoder=OneHotEncoder()
    encoder.fit(data)
    train_data=encoder.transform(data)
    return train_data

In [38]:
train_data=hot_encode(X)
X_train=train_data[:train_size,:]
X_val=train_data[train_size:,:]
y_train=y[:train_size]
y_val=y[train_size:]

In [39]:
y_train=np.log(y_train.astype(np.int))
y_val=np.log(y_val.astype(np.int))
y_max=max(np.max(y_train),np.max(y_val))
y_train=y_train/y_max
y_val=y_val/y_max

In [40]:
X_train, y_train = sample(X_train, y_train, 200000)  # Simulate data sparsity
print("Number of samples used for training: " + str(y_train.shape[0]))

Number of samples used for training: 200000


In [41]:
print('The shape of the training data is:',X_train.shape)

The shape of the training data is: (200000, 1183)


In [42]:
model=RandomForestRegressor(n_estimators = 200, max_depth = 35, min_samples_split = 2, min_samples_leaf = 1,n_jobs=-1,verbose=1)
model.fit(X_train,y_train)
y_pred=model.predict(X_val)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  7.4min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.8s finished


In [43]:
print('MAPE score is:',MAPE(y_val,y_pred))
print('R2 score is:',r2_score(y_val,y_pred))

MAPE score is: 0.026277695002020946
R2 score is: 0.4794118407199113


In [44]:
result={}
result['Without EE']={'Random Forest':MAPE(y_val,y_pred)}

Here, we are noticing that the the MAPE score is increasing and the R2 score is decreasing quite a lot. I am not able to completely understand the reason behind this.

### 2.2 Boosted Trees

For boosting, we will use XGBoost for regression
1. We will create a DMatrix from XGB for this - because we want to define a param_grid here. 
  * Again look at the parameters from the paper
2. The DMatrix should be provided with X_train and label as y_train
3. Parameters will be as follows:
  * 'nthread': -1,
  * 'max_depth': 7,
  * 'eta': 0.02,
  * 'silent': 1,
  * 'objective': 'reg:linear',
  * 'colsample_bytree': 0.7,
  * 'subsample': 0.7
  * num_round = 3000

4. Train the model

5. Note xgb.DMatrix needs features from Xtrain and Xval while predicting. Hence define:
```
feature_Xtr = xgb.DMatrix(X_train)
feature_Xval = xgb.DMatrix(X_val)
```
5. Predict on feature_Xtr and feature_Xval 
6. Calculate MAPE for both



Look at XGBoost [documentation](https://xgboost.readthedocs.io/en/latest/python/python_intro.html) for each parameter information

In [45]:
#your code here
dtrain=xgb.DMatrix(X_train,label=y_train)
num_round=3000
param={'nthread': -1,'max_depth': 7,'eta': 0.02,'silent': 1,'objective': 'reg:linear','colsample_bytree': 0.7,'subsample': 0.7}
best=xgb.train(param,dtrain,num_round)

In [46]:
feature_Xtr = xgb.DMatrix(X_train)
feature_Xval = xgb.DMatrix(X_val)
ytrain_pred=best.predict(feature_Xtr)
yval_pred=best.predict(feature_Xval)

In [47]:
print('Train MAPE score is:',MAPE(y_train,ytrain_pred))
print('Validation MAPE score is:',MAPE(y_val,yval_pred))

Train MAPE score is: 0.019496172612389288
Validation MAPE score is: 0.02025831575180082


In [49]:
print('Validation R2 score is:',r2_score(y_val,yval_pred))

Validation R2 score is: 0.6993665897072532


In [66]:
result['Without EE']['Gradient Boosted Tree']=MAPE(y_val,yval_pred)

### 2.3 Multi Layer Perceptron

Define a Sequential model with the following:
(Read the Part VI Part A Neural Networks)

1. Dense Layer - 1000 neurons, keep the kernel_initializer as uniform, with activaation as relu
2. Dense Layer - 500 neurons, keep the kernel_initializer as uniform, with activaation as relu
3. Final dense layer with 1 neuron, and activation as sigmoid
4. Compile the model on mean absolute error and optimizer as adam
5. Fit the model on 10 epochs and batch size as 128, find the MAPE 

In [51]:
#Build the model
#your code here
model = Sequential()
model.add(Dense(1000,activation='relu'))
model.add(Dense(500,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam', loss='mae')
# This builds the model for the first time:

In [ ]:
#we will rescale our y_train for the model
#the reason for this is mentioned in the paper in the same section
# to see this change you can plot the log(Sale) vs log(Sale_max) and see how it varies
max_log_y = max(np.max(np.log(y_train.astype(np.int))), np.max(np.log(y_val.astype(np.int))))
fitting_y = np.log(y_train.astype(np.int)) / max_log_y
#Since we have already done this, there is no need to do it again

In [52]:
#fit your model 
#your code here
model.fit(X_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
1563/1563 [==============================] - 17s 10ms/step - loss: 0.0185
Epoch 2/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0082
Epoch 3/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0071
Epoch 4/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0064
Epoch 5/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0058
Epoch 6/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0054
Epoch 7/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0050
Epoch 8/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0047
Epoch 9/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0044
Epoch 10/10
1563/1563 [==============================] - 16s 10ms/step - loss: 0.0041


In [53]:
y_pred=model.predict(X_val)

In [55]:
print('R2 score for the Neural Net model is:',r2_score(y_val,y_pred))

R2 score for the Neural Net model is: 0.8880116849854446


In [64]:
#predict and mape calculation
#your code here
y_pred=model.predict(X_val)
print('Validation MAPE score is:',MAPE(y_val,y_pred.flatten()))

Validation MAPE score is: 0.011896977798439066


In [67]:
result['Without EE']['Neural Network']=MAPE(y_val,y_pred.flatten())

# You are done with Part 2!!
Print out the MAPE values for all models, you will need this in hand while working on Part 3 for comparing!